In [11]:
import pandas as pd
import ast

df = pd.read_csv('Copy of TRADES_CopyTr_90D_ROI.csv')

In [5]:
df.head()

,Port_IDs,Trade_History
0,3925368433214965504,"[{'time': 1718899656000, 'symbol': 'SOLUSDT', ..."
1,4002413037164645377,"[{'time': 1718980078000, 'symbol': 'NEARUSDT',..."
2,3923766029921022977,"[{'time': 1718677164000, 'symbol': 'ETHUSDT', ..."
3,3994879592543698688,"[{'time': 1718678214000, 'symbol': 'ETHUSDT', ..."
4,3926423286576838657,"[{'time': 1718979615000, 'symbol': 'ETHUSDT', ..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Port_IDs       150 non-null    int64 
 1   Trade_History  149 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB


In [12]:
df['Trade_History'] = df['Trade_History'].str.replace('nan', 'None', regex=True)

df['Trade_History'] = df['Trade_History'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else np.nan)

df = df.explode('Trade_History')

trade_df = pd.json_normalize(df['Trade_History'])

df = df.drop(columns=['Trade_History']).reset_index(drop=True)
trade_df = trade_df.reset_index(drop=True)

df = pd.concat([df, trade_df], axis=1)

df['time'] = pd.to_datetime(df['time'], unit='ms')

In [14]:
df.head()

,Port_IDs,time,symbol,side,price,fee,feeAsset,quantity,quantityAsset,realizedProfit,realizedProfitAsset,baseAsset,qty,positionSide,activeBuy
0,3925368433214965504,2024-06-20 16:07:36,SOLUSDT,BUY,132.53700,-0.994027,USDT,1988.05500,USDT,0.0,USDT,SOL,15.0,LONG,True
1,3925368433214965504,2024-06-20 16:06:58,DOGEUSDT,BUY,0.12182,-0.279796,USDT,1398.98088,USDT,0.0,USDT,DOGE,11484.0,LONG,False
2,3925368433214965504,2024-06-20 16:06:58,DOGEUSDT,BUY,0.12182,-0.039494,USDT,197.47022,USDT,0.0,USDT,DOGE,1621.0,LONG,False
3,3925368433214965504,2024-06-20 16:06:56,DOGEUSDT,BUY,0.12182,-0.008284,USDT,16.56752,USDT,0.0,USDT,DOGE,136.0,LONG,True
4,3925368433214965504,2024-06-20 16:06:56,DOGEUSDT,BUY,0.12182,-0.046109,USDT,92.21774,USDT,0.0,USDT,DOGE,757.0,LONG,True


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211278 entries, 0 to 211277
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Port_IDs             211278 non-null  int64         
 1   time                 211277 non-null  datetime64[ns]
 2   symbol               211277 non-null  object        
 3   side                 211277 non-null  object        
 4   price                211277 non-null  float64       
 5   fee                  211277 non-null  float64       
 6   feeAsset             211277 non-null  object        
 7   quantity             211277 non-null  float64       
 8   quantityAsset        211277 non-null  object        
 9   realizedProfit       211277 non-null  float64       
 10  realizedProfitAsset  211277 non-null  object        
 11  baseAsset            211277 non-null  object        
 12  qty                  211277 non-null  float64       
 13  positionSide  

In [18]:
df.isnull().sum()

,0
Port_IDs,0
time,1
symbol,1
side,1
price,1
fee,1
feeAsset,1
quantity,1
quantityAsset,1
realizedProfit,1


## **Since the no.of null value is less than 5% of the data,we should drop it.**

In [19]:
df = df.dropna()

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 211277 entries, 0 to 211277
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Port_IDs             211277 non-null  int64         
 1   time                 211277 non-null  datetime64[ns]
 2   symbol               211277 non-null  object        
 3   side                 211277 non-null  object        
 4   price                211277 non-null  float64       
 5   fee                  211277 non-null  float64       
 6   feeAsset             211277 non-null  object        
 7   quantity             211277 non-null  float64       
 8   quantityAsset        211277 non-null  object        
 9   realizedProfit       211277 non-null  float64       
 10  realizedProfitAsset  211277 non-null  object        
 11  baseAsset            211277 non-null  object        
 12  qty                  211277 non-null  float64       
 13  positionSide       

In [21]:
df['trade_type'] = df['side'] + '_' + df['positionSide'].str.lower()

df['PnL'] = df['realizedProfit']

df['ROI'] = df['PnL'] / df['quantity']

df['Sharpe Ratio'] = df['PnL'].mean() / df['PnL'].std()

df['cumulative_PnL'] = df.groupby('Port_IDs')['PnL'].cumsum()
df['peak'] = df.groupby('Port_IDs')['cumulative_PnL'].cummax()
df['drawdown'] = df['peak'] - df['cumulative_PnL']
df['MDD'] = df.groupby('Port_IDs')['drawdown'].max()

df['win'] = df['PnL'] > 0
win_rate = df.groupby('Port_IDs')['win'].mean()

total_positions = df.groupby('Port_IDs').size()

metrics = df.groupby('Port_IDs').agg({
    'PnL': 'sum',
    'ROI': 'mean',
    'Sharpe Ratio': 'mean',
    'MDD': 'max',
    'win': 'mean',
    'trade_type': 'count'
}).rename(columns={'win': 'Win Rate', 'trade_type': 'Total Positions'})

metrics['Win Positions'] = metrics['Win Rate'] * metrics['Total Positions']

In [22]:
normalized_metrics = (metrics - metrics.min()) / (metrics.max() - metrics.min())

weights = {
    'PnL': 0.3,
    'ROI': 0.2,
    'Sharpe Ratio': 0.2,
    'MDD': -0.1,
    'Win Rate': 0.1,
    'Win Positions': 0.1
}

normalized_metrics['Score'] = sum(normalized_metrics[col] * weight for col, weight in weights.items())

ranked_accounts = normalized_metrics.sort_values(by='Score', ascending=False)

In [34]:
ranked_accounts.to_csv('ranked_accounts.csv')

report = """
Analysis Report:

Methodology:

1. Data Cleaning: The dataset was cleaned by converting the `Trade_History` column from a string to a list of dictionaries and then normalizing it into separate columns.
2. Feature Engineering: Trades were classified, and financial metrics such as PnL, ROI, Sharpe Ratio, MDD, Win Rate, and Total Positions were calculated.
3. Ranking Algorithm: Metrics were normalized, weighted, and combined into a single score to rank the accounts.

Findings:

- The top 20 accounts based on the weighted score are:

  1 3672754654734989568
  2	3733192481840423936
  3	3768170840939476993
  4	3784403294629753856
  5	3786761687746711808
  6	3788465932399412480
  7	3818233536529843712
  8	3819545518395756033
  9	3826087012661391104
 10	3858510226868015873
 11	3865845304835489536
 12	3878631538480067329
 13	3879821005658659073
 14	3880920727367689984
 15	3886752488982104320
 16	3887577207880438784
 17	3891020560590657281
 18	3907081197088384000
 19	3910887259807777281
 20	3910983669796782848

Assumptions:

- The risk-free rate for the Sharpe Ratio calculation was assumed to be 0.
- Weights for the scoring system were chosen based on typical importance in financial analysis.
"""

print(report)


Analysis Report:

Methodology:

1. Data Cleaning: The dataset was cleaned by converting the `Trade_History` column from a string to a list of dictionaries and then normalizing it into separate columns.
2. Feature Engineering: Trades were classified, and financial metrics such as PnL, ROI, Sharpe Ratio, MDD, Win Rate, and Total Positions were calculated.
3. Ranking Algorithm: Metrics were normalized, weighted, and combined into a single score to rank the accounts.

Findings:

- The top 20 accounts based on the weighted score are:

  1 3672754654734989568
  2	3733192481840423936
  3	3768170840939476993
  4	3784403294629753856
  5	3786761687746711808
  6	3788465932399412480
  7	3818233536529843712
  8	3819545518395756033
  9	3826087012661391104
 10	3858510226868015873
 11	3865845304835489536
 12	3878631538480067329
 13	3879821005658659073
 14	3880920727367689984
 15	3886752488982104320
 16	3887577207880438784
 17	3891020560590657281
 18	3907081197088384000
 19	3910887259807777281
 20	391

In [26]:
rankers=pd.read_csv("ranked_accounts.csv")

In [27]:
rankers.head(20)

,Port_IDs,PnL,ROI,Sharpe Ratio,MDD,Win Rate,Total Positions,Win Positions,Score
0,3672754654734989568,0.153100,0.067268,0.5,NaN,0.551995,0.075770,0.072589,NaN
1,3733192481840423936,0.181049,0.061073,0.5,NaN,1.000000,0.111184,0.191151,NaN
2,3768170840939476993,0.149271,0.510963,0.5,NaN,0.533971,0.000000,0.002074,NaN
3,3784403294629753856,0.176281,0.038947,0.5,NaN,0.376663,0.994235,0.632216,NaN
4,3786761687746711808,0.148813,0.059994,0.5,NaN,0.562189,0.011201,0.012789,NaN
5,3788465932399412480,0.311903,0.045499,0.0,NaN,0.555777,0.503212,0.473211,NaN
6,3818233536529843712,0.222493,0.050553,0.5,NaN,0.422617,0.231758,0.166609,NaN
7,3819545518395756033,0.158002,0.044405,0.5,NaN,0.476385,0.008895,0.008987,NaN
8,3826087012661391104,0.152697,1.000000,0.5,NaN,0.726793,0.015483,0.021777,NaN
9,3858510226868015873,0.143051,0.032641,0.5,NaN,0.303977,0.161753,0.083996,NaN


In [28]:
print(ranked_accounts.head(20).index.tolist())

[3672754654734989568, 3733192481840423936, 3768170840939476993, 3784403294629753856, 3786761687746711808, 3788465932399412480, 3818233536529843712, 3819545518395756033, 3826087012661391104, 3858510226868015873, 3865845304835489536, 3878631538480067329, 3879821005658659073, 3880920727367689984, 3886752488982104320, 3887577207880438784, 3891020560590657281, 3907081197088384000, 3910887259807777281, 3910983669796782848]


In [29]:
top_20_accounts = ranked_accounts.head(20).index.tolist()

In [30]:
df_top_20 = pd.DataFrame({'Account_ID': top_20_accounts})

In [36]:
print(df_top_20)

             Account_ID
0   3672754654734989568
1   3733192481840423936
2   3768170840939476993
3   3784403294629753856
4   3786761687746711808
5   3788465932399412480
6   3818233536529843712
7   3819545518395756033
8   3826087012661391104
9   3858510226868015873
10  3865845304835489536
11  3878631538480067329
12  3879821005658659073
13  3880920727367689984
14  3886752488982104320
15  3887577207880438784
16  3891020560590657281
17  3907081197088384000
18  3910887259807777281
19  3910983669796782848
